In [2]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(tf.__version__)
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

1.14.0


In [3]:
vocab_size = 5000
embedding_dim = 64
max_length = 5000
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8

import sys
import csv
csv.field_size_limit(sys.maxsize)


articles = []
labels = []

with open("all.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        #print(row[0])
        article = row[1]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)
print(len(labels))
print(len(articles))
print (labels[2])


145
145
 Deshacop


In [4]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_articles))
print(len(train_labels))
print(len(validation_articles))
print(len(validation_labels))

116
116
116
29
29


In [5]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

{'<OOV>': 1,
 'ldrgetprocedureaddress': 2,
 'ntclose': 3,
 'ntdelayexecution': 4,
 'module32nextw': 5,
 'ntallocatevirtualmemory': 6,
 'ntquerydirectoryfile': 7,
 'ldrloaddll': 8,
 'process32nextw': 9,
 'findfirstfileexw': 10}

In [6]:
train_sequences = tokenizer.texts_to_sequences(train_articles)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(len(train_sequences[0]))
print(len(train_padded[0]))


print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

4000
5000
3641
5000
4659
5000


In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Getting trigrams  
vectorizer = CountVectorizer(ngram_range = (3,3)) 
X1 = vectorizer.fit_transform(train_articles)  
features = (vectorizer.get_feature_names()) 
print("\n\nFeatures : \n", features) 
print("\n\nX1 : \n", X1.toarray()) 



Features : 
 ['closesocket coinitializeex cocreateinstance', 'closesocket coinitializeex coinitializeex', 'closesocket coinitializeex messageboxtimeoutw', 'closesocket coinitializeex ntclose', 'closesocket ldrgetprocedureaddress ldrloaddll', 'closesocket ntallocatevirtualmemory coinitializeex', 'closesocket ntclose couninitialize', 'closesocket ntclose ntcreatefile', 'closesocket ntdelayexecution cryptencrypt', 'closesocket ntprotectvirtualmemory ldrgetprocedureaddress', 'closesocket socket ioctlsocket', 'cocreateinstance cocreateinstance couninitialize', 'cocreateinstance cocreateinstance findresourcew', 'cocreateinstance cocreateinstance ldrgetdllhandle', 'cocreateinstance cocreateinstance ldrloaddll', 'cocreateinstance cocreateinstance ntallocatevirtualmemory', 'cocreateinstance cocreateinstance ntfreevirtualmemory', 'cocreateinstance cocreateinstance ntopenkey', 'cocreateinstance cocreateinstance regopenkeyexa', 'cocreateinstance cocreateinstance uuidcreate', 'cocreateinstance co

In [7]:
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

29
(29, 5000)


In [8]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))
print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)

[1]
[3]
[3]
(116, 1)
[3]
[1]
[2]
(29, 1)


In [17]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_article(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
print(decode_article(train_padded[10]))
print('---')
print(train_articles[0])

ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetproced

In [14]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(6, activation='softmax')
])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
____________________________________________

In [10]:
print(set(labels))

{' Cryptodef', ' Loktrom ', ' Loktrom', ' Deshacop', ' Gpcode'}


In [15]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Train on 116 samples, validate on 29 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
116/116 - 40s - loss: 1.7744 - acc: 0.3017 - val_loss: 1.7581 - val_acc: 0.2759
Epoch 2/10
116/116 - 28s - loss: 1.7281 - acc: 0.3534 - val_loss: 1.7130 - val_acc: 0.2759
Epoch 3/10
116/116 - 85s - loss: 1.6551 - acc: 0.3276 - val_loss: 1.6343 - val_acc: 0.2414
Epoch 4/10
116/116 - 29s - loss: 1.5328 - acc: 0.3276 - val_loss: 1.5499 - val_acc: 0.2414
Epoch 5/10
116/116 - 24s - loss: 1.4198 - acc: 0.3276 - val_loss: 1.5316 - val_acc: 0.2414
Epoch 6/10
116/116 - 22s - loss: 1.3725 - acc: 0.3707 - val_loss: 1.4260 - val_acc: 0.4828
Epoch 7/10
116/116 - 22s - loss: 1.3152 - acc: 0.5690 - val_loss: 1.3827 - val_acc: 0.3448
Epoch 8/10
116/116 - 22s - loss: 1.2809 - acc: 0.3793 - val_loss: 1.3235 - val_acc: 0.3103
Epoch 9/10
116/116 - 22s - loss: 1.2014 - acc: 0.4655 - val_loss: 1.2669 - val_acc: 0.4138
Epoch 10/10
116/116 - 22s - loss: 1.1175 -

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# determine the number of input features
n_features = train_padded.shape[1]
# define model
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
#model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(6, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# fit the model
#model.fit(train_padded,training_label_seq, epochs=50, batch_size=32, verbose=0)
history = model.fit(train_padded, training_label_seq, epochs=150, validation_data=(validation_padded, validation_label_seq), verbose=2)
# evaluate the model
loss, acc = model.evaluate(validation_padded, validation_label_seq, verbose=0)
print('Test Accuracy: %.3f' % acc)

Train on 116 samples, validate on 29 samples
Epoch 1/150
116/116 - 0s - loss: 10.1882 - acc: 0.2414 - val_loss: 2.8356 - val_acc: 0.3793
Epoch 2/150
116/116 - 0s - loss: 2.0671 - acc: 0.4310 - val_loss: 1.9124 - val_acc: 0.4138
Epoch 3/150
116/116 - 0s - loss: 1.6612 - acc: 0.5172 - val_loss: 2.0227 - val_acc: 0.4483
Epoch 4/150
116/116 - 0s - loss: 1.3676 - acc: 0.5517 - val_loss: 2.0646 - val_acc: 0.3793
Epoch 5/150
116/116 - 0s - loss: 1.1388 - acc: 0.5948 - val_loss: 2.0416 - val_acc: 0.3793
Epoch 6/150
116/116 - 0s - loss: 1.0210 - acc: 0.5862 - val_loss: 1.8604 - val_acc: 0.5172
Epoch 7/150
116/116 - 0s - loss: 0.9911 - acc: 0.6034 - val_loss: 1.8688 - val_acc: 0.5172
Epoch 8/150
116/116 - 0s - loss: 0.9409 - acc: 0.6379 - val_loss: 1.9322 - val_acc: 0.4828
Epoch 9/150
116/116 - 0s - loss: 0.8682 - acc: 0.7241 - val_loss: 2.0549 - val_acc: 0.5862
Epoch 10/150
116/116 - 0s - loss: 0.8473 - acc: 0.6810 - val_loss: 2.3402 - val_acc: 0.5517
Epoch 11/150
116/116 - 0s - loss: 0.8341 - 

Epoch 90/150
116/116 - 0s - loss: 0.3423 - acc: 0.8621 - val_loss: 4.6174 - val_acc: 0.4828
Epoch 91/150
116/116 - 0s - loss: 0.3184 - acc: 0.8966 - val_loss: 4.5463 - val_acc: 0.5172
Epoch 92/150
116/116 - 0s - loss: 0.3157 - acc: 0.8793 - val_loss: 4.6125 - val_acc: 0.5517
Epoch 93/150
116/116 - 0s - loss: 0.2839 - acc: 0.9052 - val_loss: 4.7964 - val_acc: 0.5517
Epoch 94/150
116/116 - 0s - loss: 0.2852 - acc: 0.8879 - val_loss: 4.3467 - val_acc: 0.5862
Epoch 95/150
116/116 - 0s - loss: 0.2590 - acc: 0.9138 - val_loss: 4.2667 - val_acc: 0.5862
Epoch 96/150
116/116 - 0s - loss: 0.2660 - acc: 0.9138 - val_loss: 4.5449 - val_acc: 0.5862
Epoch 97/150
116/116 - 0s - loss: 0.2577 - acc: 0.9224 - val_loss: 4.8182 - val_acc: 0.5172
Epoch 98/150
116/116 - 0s - loss: 0.2549 - acc: 0.9310 - val_loss: 4.9844 - val_acc: 0.5172
Epoch 99/150
116/116 - 0s - loss: 0.2478 - acc: 0.9310 - val_loss: 5.1613 - val_acc: 0.5517
Epoch 100/150
116/116 - 0s - loss: 0.2528 - acc: 0.9138 - val_loss: 4.7830 - val

In [13]:
import matplotlib.pyplot as plt
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

KeyError: 'accuracy'

In [40]:
txt = ["ReadProcessMemory ReadProcessMemory ReadProcessMemory ReadProcessMemory ReadProcessMemory LdrLoadDll LdrGetProcedureAddress RegCreateKeyExA RegQueryValueExA RegCloseKey RegOpenKeyExA RegCreateKeyExA RegCloseKey RegCreateKeyExA RegSetValueExA RegSetValueExA RegSetValueExA RegSetValueExA RegSetValueExA RegSetValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA LdrGetProcedureAddress NtAllocateVirtualMemory LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress EnumWindows LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtAllocateVirtualMemory LdrGetProcedureAddress RtlDecompressBuffer LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress GetSystemTimeAsFileTime LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey GetSystemInfo LdrGetDllHandle LdrGetProcedureAddress RegOpenKeyExW LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress NtProtectVirtualMemory LdrLoadDll LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtClose NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory GetNativeSystemInfo GetSystemTimeAsFileTime LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress CreateThread LdrGetProcedureAddress GetSystemTimeAsFileTime NtAllocateVirtualMemory NtFreeVirtualMemory NtAllocateVirtualMemory LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetDllHandle NtAllocateVirtualMemory NtAllocateVirtualMemory SetUnhandledExceptionFilter CoInitializeEx NtClose NtClose SHGetFolderPathW NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution LdrGetDllHandle LdrGetProcedureAddress CoInitializeEx NtCreateMutant CoCreateInstance RegOpenKeyExW GetSystemInfo NtCreateSection NtClose NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtFreeVirtualMemory NtAllocateVirtualMemory LdrLoadDll LdrGetProcedureAddress IsDebuggerPresent GetSystemMetrics NtMapViewOfSection LdrGetDllHandle LdrGetProcedureAddress NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory GetSystemMetrics GetSystemMetrics GetSystemMetrics NtAllocateVirtualMemory GetSystemMetrics GetSystemMetrics LdrLoadDll LdrGetProcedureAddress GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics NtOpenProcess NtClose NtClose NtClose LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress CoCreateInstance CoCreateInstance LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW DeleteFileW NtCreateFile SetFilePointer NtReadFile NtClose LookupPrivilegeValueW NtClose NtClose NtDelayExecution NtDelayExecution NtDelayExecution NtCreateFile NtClose CopyFileW CreateProcessInternalW GetShortPathNameW RegOpenKeyExA NtDuplicateObject GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW LoadStringW GetSystemWindowsDirectoryW GetSystemDirectoryW RegOpenKeyExW LdrLoadDll LdrGetProcedureAddress RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey NtCreateMutant NtCreateMutant GetNativeSystemInfo GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW NtClose LdrLoadDll LdrGetProcedureAddress NtDuplicateObject NtAllocateVirtualMemory LdrGetProcedureAddress NtDuplicateObject GetVolumeNameForVolumeMountPointW NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey NtDuplicateObject NtClose GetVolumeNameForVolumeMountPointW NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey NtClose RegQueryValueExW RegCloseKey NtDuplicateObject GetVolumeNameForVolumeMountPointW NtClose NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose LdrGetProcedureAddress LdrUnloadDll CreateProcessInternalW ShellExecuteExW LdrGetDllHandle LdrGetDllHandle NtTerminateProcess NtTerminateProcess NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtUnmapViewOfSection NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtUnmapViewOfSection NtClose NtClose NtClose LdrUnloadDll LdrGetProcedureAddress NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtUnmapViewOfSection NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtClose NtClose NtClose NtClose NtClose RegCloseKey RegCloseKey RegCloseKey RegCloseKey NtClose RegCloseKey RegCloseKey RegCloseKey RegCloseKey RegCloseKey RegCloseKey RegCloseKey RegCloseKey NtClose LdrGetProcedureAddress NtClose LdrGetProcedureAddress RegCloseKey RegCloseKey RegCloseKey RegCloseKey NtClose NtClose NtClose NtClose NtClose NtUnmapViewOfSection NtUnmapViewOfSection NtClose NtUnmapViewOfSection NtClose RegCloseKey RegCloseKey NtClose NtClose LdrGetDllHandle LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress LdrUnloadDll NtOpenKey NtQueryValueKey NtClose NtClose NtClose LdrUnloadDll NtClose NtTerminateProcess ReadProcessMemory ReadProcessMemory ReadProcessMemory ReadProcessMemory ReadProcessMemory LdrLoadDll LdrGetProcedureAddress RegCreateKeyExA RegQueryValueExA RegCloseKey RegOpenKeyExA RegCreateKeyExA RegCloseKey RegCreateKeyExA RegSetValueExA RegSetValueExA RegSetValueExA RegSetValueExA RegSetValueExA RegSetValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA RegQueryValueExA LdrGetProcedureAddress NtAllocateVirtualMemory LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress EnumWindows LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtAllocateVirtualMemory LdrGetProcedureAddress RtlDecompressBuffer LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress GetSystemTimeAsFileTime LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey GetSystemInfo LdrGetDllHandle LdrGetProcedureAddress RegOpenKeyExW LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress NtProtectVirtualMemory LdrLoadDll LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtClose NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory GetNativeSystemInfo GetSystemTimeAsFileTime LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress CreateThread LdrGetProcedureAddress GetSystemTimeAsFileTime NtAllocateVirtualMemory NtFreeVirtualMemory NtAllocateVirtualMemory LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetDllHandle NtAllocateVirtualMemory NtAllocateVirtualMemory SetUnhandledExceptionFilter CoInitializeEx NtClose NtClose SHGetFolderPathW NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution LdrGetDllHandle LdrGetProcedureAddress CoInitializeEx NtCreateMutant CoCreateInstance RegOpenKeyExW GetSystemInfo NtCreateSection NtClose NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtFreeVirtualMemory NtAllocateVirtualMemory LdrLoadDll LdrGetProcedureAddress IsDebuggerPresent GetSystemMetrics NtMapViewOfSection LdrGetDllHandle LdrGetProcedureAddress NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory GetSystemMetrics GetSystemMetrics GetSystemMetrics NtAllocateVirtualMemory GetSystemMetrics GetSystemMetrics LdrLoadDll LdrGetProcedureAddress GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics NtOpenProcess NtClose NtClose NtClose LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress CoCreateInstance CoCreateInstance LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW DeleteFileW NtCreateFile SetFilePointer NtReadFile NtClose LookupPrivilegeValueW NtClose NtClose NtDelayExecution NtDelayExecution NtDelayExecution NtCreateFile NtClose NtCreateMutant RegCreateKeyExW RegCreateKeyExW RegQueryValueExW LdrLoadDll LdrLoadDll LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress NtCreateFile NtSetInformationFile NtSetInformationFile NtSetInformationFile RegOpenKeyExW NtWriteFile NtAllocateVirtualMemory NtReadFile NtAllocateVirtualMemory NtCreateFile NtSetInformationFile NtSetInformationFile NtSetInformationFile NtWriteFile NtReadFile LdrUnloadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress CryptAcquireContextW LdrGetProcedureAddress LdrGetProcedureAddress CryptAcquireContextW LdrUnloadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress CreateToolhelp32Snapshot NtMapViewOfSection NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtMapViewOfSection NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtMapViewOfSection NtUnmapViewOfSection Process32FirstW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Thread32First Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Module32FirstW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW NtClose LdrUnloadDll GlobalMemoryStatus RegSetValueExW RegCloseKey NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory RegCreateKeyExW RegQueryValueExW RegCloseKey GetSystemTimeAsFileTime RegCreateKeyExW RegSetValueExW RegCloseKey NtAllocateVirtualMemory NtAllocateVirtualMemory RegCreateKeyExW RegSetValueExW RegCloseKey RegCreateKeyExW RegSetValueExW RegCloseKey RegCreateKeyExW RegSetValueExW RegCloseKey CreateThread NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetProcedureAddress CryptAcquireContextA LdrGetDllHandle LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetProcedureAddress CryptDecrypt LdrGetDllHandle LdrGetProcedureAddress LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrLoadDll LdrGetProcedureAddress OleInitialize LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExA LdrGetProcedureAddress LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA CreateActCtxW GetSystemWindowsDirectoryW CreateActCtxW LdrGetDllHandle LdrGetProcedureAddress LdrLoadDll LdrLoadDll LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW LdrLoadDll LdrGetProcedureAddress RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW NtOpenProcess NtClose LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptAcquireContextA LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle CryptDecrypt LdrGetDllHandle LdrGetDllHandle LdrGetDllHandle LdrGetDllHa"]
#txt = ["NtAllocateVirtualMemory LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress EnumWindows LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtAllocateVirtualMemory LdrGetProcedureAddress RtlDecompressBuffer LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress GetSystemTimeAsFileTime LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress NtProtectVirtualMemory LdrLoadDll LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtClose NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory GetNativeSystemInfo GetSystemTimeAsFileTime LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress CreateThread LdrGetProcedureAddress NtClose NtClose GetSystemTimeAsFileTime NtAllocateVirtualMemory NtFreeVirtualMemory NtAllocateVirtualMemory LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetDllHandle NtAllocateVirtualMemory NtAllocateVirtualMemory SetUnhandledExceptionFilter LoadStringW LoadStringW FindResourceExW LoadResource FindResourceExW LoadResource FindResourceExW LoadResource FindResourceExW LoadResource CoInitializeEx NtClose NtClose SHGetFolderPathW NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution LdrGetDllHandle LdrGetProcedureAddress CoInitializeEx NtCreateMutant CoCreateInstance RegOpenKeyExW GetSystemInfo NtCreateSection NtClose NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtFreeVirtualMemory NtAllocateVirtualMemory LdrLoadDll LdrGetProcedureAddress IsDebuggerPresent GetSystemMetrics NtMapViewOfSection LdrGetDllHandle LdrGetProcedureAddress NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory GetSystemMetrics GetSystemMetrics GetSystemMetrics NtAllocateVirtualMemory GetSystemMetrics GetSystemMetrics LdrLoadDll LdrGetProcedureAddress GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics NtOpenProcess NtClose NtClose NtClose LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress CoCreateInstance CoCreateInstance LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW DeleteFileW NtCreateFile SetFilePointer NtReadFile NtClose LookupPrivilegeValueW NtClose NtClose NtDelayExecution NtDelayExecution NtDelayExecution NtCreateFile NtClose CopyFileW CreateProcessInternalW GetShortPathNameW RegOpenKeyExA NtDuplicateObject GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW LoadStringW GetSystemWindowsDirectoryW GetSystemDirectoryW RegOpenKeyExW LdrLoadDll LdrGetProcedureAddress RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey NtCreateMutant NtCreateMutant GetNativeSystemInfo GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW NtClose LdrLoadDll LdrGetProcedureAddress NtDuplicateObject NtAllocateVirtualMemory LdrGetProcedureAddress NtDuplicateObject GetVolumeNameForVolumeMountPointW NtClose NtAllocateVirtualMemory RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey NtDuplicateObject NtClose GetVolumeNameForVolumeMountPointW NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey NtClose RegQueryValueExW RegCloseKey NtDuplicateObject GetVolumeNameForVolumeMountPointW NtClose NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose LdrGetProcedureAddress LdrUnloadDll CreateProcessInternalW ShellExecuteExW LdrGetDllHandle LdrGetDllHandle NtTerminateProcess NtTerminateProcess NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtUnmapViewOfSection NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtUnmapViewOfSection NtClose NtClose NtClose LdrUnloadDll LdrGetProcedureAddress NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtClose NtUnmapViewOfSection NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtClose NtClose NtClose NtClose NtClose RegCloseKey RegCloseKey RegCloseKey RegCloseKey NtClose RegCloseKey RegCloseKey RegCloseKey RegCloseKey RegCloseKey RegCloseKey RegCloseKey RegCloseKey NtClose NtClose NtClose NtClose NtClose LdrGetProcedureAddress RegCloseKey RegCloseKey RegCloseKey RegCloseKey NtClose NtClose NtClose NtClose NtClose NtUnmapViewOfSection LdrGetDllHandle LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress NtClose NtClose LdrUnloadDll NtUnmapViewOfSection NtClose NtUnmapViewOfSection NtClose RegCloseKey RegCloseKey NtClose NtClose NtClose LdrGetProcedureAddress NtClose LdrUnloadDll NtOpenKey NtQueryValueKey NtClose NtTerminateProcess NtAllocateVirtualMemory LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress EnumWindows LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtAllocateVirtualMemory LdrGetProcedureAddress RtlDecompressBuffer LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress GetSystemTimeAsFileTime LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress NtProtectVirtualMemory LdrLoadDll LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtClose NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory GetNativeSystemInfo GetSystemTimeAsFileTime LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress CreateThread LdrGetProcedureAddress NtClose NtClose GetSystemTimeAsFileTime NtAllocateVirtualMemory NtFreeVirtualMemory NtAllocateVirtualMemory LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetDllHandle NtAllocateVirtualMemory NtAllocateVirtualMemory SetUnhandledExceptionFilter LoadStringW LoadStringW FindResourceExW LoadResource FindResourceExW LoadResource FindResourceExW LoadResource FindResourceExW LoadResource CoInitializeEx NtClose NtClose SHGetFolderPathW NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution NtDelayExecution LdrGetDllHandle LdrGetProcedureAddress CoInitializeEx NtCreateMutant CoCreateInstance RegOpenKeyExW GetSystemInfo NtCreateSection NtClose NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtFreeVirtualMemory NtAllocateVirtualMemory LdrLoadDll LdrGetProcedureAddress IsDebuggerPresent GetSystemMetrics NtMapViewOfSection LdrGetDllHandle LdrGetProcedureAddress NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory GetSystemMetrics GetSystemMetrics GetSystemMetrics NtAllocateVirtualMemory GetSystemMetrics GetSystemMetrics LdrLoadDll LdrGetProcedureAddress GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics NtOpenProcess NtClose NtClose NtClose LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress CoCreateInstance CoCreateInstance LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW SHGetFolderPathW DeleteFileW NtCreateFile SetFilePointer NtReadFile NtClose LookupPrivilegeValueW NtClose NtClose NtDelayExecution NtDelayExecution NtDelayExecution NtCreateFile NtClose NtCreateMutant RegCreateKeyExW RegCreateKeyExW RegQueryValueExW LdrLoadDll LdrLoadDll LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress NtCreateFile NtSetInformationFile NtSetInformationFile NtSetInformationFile RegOpenKeyExW NtWriteFile NtAllocateVirtualMemory NtReadFile NtAllocateVirtualMemory NtAllocateVirtualMemory NtCreateFile NtSetInformationFile NtSetInformationFile NtSetInformationFile NtWriteFile NtReadFile LdrUnloadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress CryptAcquireContextW LdrGetProcedureAddress LdrGetProcedureAddress CryptAcquireContextW LdrUnloadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress CreateToolhelp32Snapshot NtMapViewOfSection NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtCreateSection NtMapViewOfSection NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory NtClose NtUnmapViewOfSection NtMapViewOfSection NtUnmapViewOfSection Process32FirstW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Process32NextW Thread32First Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Thread32Next Module32FirstW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW Module32NextW NtClose LdrUnloadDll GlobalMemoryStatus RegSetValueExW RegCloseKey NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory RegCreateKeyExW RegQueryValueExW RegCloseKey GetSystemTimeAsFileTime RegCreateKeyExW RegSetValueExW RegCloseKey NtAllocateVirtualMemory NtAllocateVirtualMemory RegCreateKeyExW RegSetValueExW RegCloseKey RegCreateKeyExW RegSetValueExW RegCloseKey RegCreateKeyExW RegSetValueExW RegCloseKey CreateThread NtCreateFile GetFileType SetFilePointer NtAllocateVirtualMemory NtWriteFile NtClose CreateThread CreateThread CreateThread NtAllocateVirtualMemory NtQuerySystemInformation NtAllocateVirtualMemory NtQuerySystemInformation NtOpenProcess NtClose NtOpenProcess LdrGetDllHandle NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose LdrLoadDll LdrGetProcedureAddress OleInitialize LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExA LdrGetProcedureAddress CreateActCtxW GetSystemWindowsDirectoryW CreateActCtxW LdrGetDllHandle LdrGetProcedureAddress LdrLoadDll LdrLoadDll LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW LdrLoadDll LdrGetProcedureAddress RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW NtDelayExecution NtQuerySystemInformation NtQuerySystemInformation NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose InternetOpenW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW WSAStartup NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory socket NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory setsockopt NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtDeviceIoControlFile NtDeviceIoControlFile NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory closesocket NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory NtDuplicateObject RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW NtDuplicateObject RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW NtDelayExecution NtAllocateVirtualMemory NtQuerySystemInformation NtAllocateVirtualMemory NtQuerySystemInformation NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose RegQueryValueExW RegCloseKey RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW NtOpenProcess NtClose RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegQueryValueExW NtDuplicateObject RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory CoInitializeEx NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW WSAStartup socket setsockopt NtDeviceIoControlFile NtDeviceIoControlFile closesocket CoInitializeEx CoInitializeEx NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory CoCreateInstance RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW NtDelayExecution NtAllocateVirtualMemory NtQuerySystemInformation NtQuerySystemInformation NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW LdrLoadDll WSAStartup NtProtectVirtualMemory LdrGetProcedureAddress NtProtectVirtualMemory LdrGetProcedureAddress NtOpenKey NtOpenKey NtOpenKey NtOpenKey NtQueryValueKey NtClose LdrGetDllHandle NtOpenFile NtClose NtOpenKey NtOpenKey NtOpenKey NtOpenKey NtClose NtAllocateVirtualMemory RegOpenKeyExW RegCloseKey GetAddrInfoW RegOpenKeyExW RegQueryValueExW RegCloseKey NtOpenKey NtQueryValueKey NtOpenKey NtQueryValueKey NtClose NtClose LdrGetProcedureAddress NtDelayExecution NtAllocateVirtualMemory NtQuerySystemInformation NtAllocateVirtualMemory NtQuerySystemInformation NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess NtClose NtOpenProcess GetAdaptersAddresses NtFreeVirtualMemory NtFreeVirtualMemory NtClose NtClose NtAllocateVirtualMemory CoCreateInstance RegOpenKeyExA RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryVa"]
#txt = ["LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory FindResourceA SizeofResource LoadResource NtOpenDirectoryObject NtOpenMutant NtCreateMutant LdrLoadDll LdrGetProcedureAddress CryptAcquireContextA LdrGetProcedureAddress CryptGenKey LdrGetProcedureAddress CryptExportKey CryptExportKey LdrGetProcedureAddress NtAllocateVirtualMemory CryptAcquireContextA LdrGetProcedureAddress NtAllocateVirtualMemory LdrGetProcedureAddress CryptEncrypt LdrGetProcedureAddress LdrGetProcedureAddress CreateThread SetErrorMode NtDelayExecution FindFirstFileExW NtClose NtOpenKey NtQueryValueKey NtClose NtOpenKey NtQueryValueKey NtClose NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer SHGetFolderPathW NtCreateFile SetFilePointer NtWriteFile NtWriteFile NtClose RegOpenKeyExA NtReadFile CryptEncrypt SetFilePointer LdrLoadDll LdrGetProcedureAddress OleInitialize LdrGetProcedureAddress LdrGetProcedureAddress SetErrorMode GetFileAttributesW SetErrorMode GetSystemDirectoryW RegOpenKeyExA RegOpenKeyExA LdrLoadDll LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW LdrGetProcedureAddress CreateActCtxW GetSystemWindowsDirectoryW CreateActCtxW LdrGetDllHandle LdrGetProcedureAddress LdrLoadDll LdrLoadDll LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW LdrLoadDll LdrGetProcedureAddress RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegCloseKey LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW NtWriteFile SetFilePointer RegOpenKeyExW LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW NtWriteFile NtClose NtOpenFile NtQueryInformationFile RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW NtSetInformationFile NtClose NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW FindFirstFileExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegOpenKeyExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW NtReadFile CryptEncrypt SetFilePointer RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW NtWriteFile SetFilePointer RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW NtWriteFile NtClose NtOpenFile NtQueryInformationFile RegQueryValueExW RegCloseKey RegCloseKey RegCloseKey RegCloseKey LdrGetProcedureAddress CoInitializeEx LdrGetProcedureAddress CoCreateInstance NtOpenSection NtOpenSection NtSetInformationFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile CreateDirectoryW NtCreateFile LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegQueryValueExW RegQueryValueExW GetSystemTimeAsFileTime GetSystemTimeAsFileTime RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW NtQueryDirectoryFile NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey LdrLoadDll LdrGetProcedureAddress RegCloseKey NtClose LdrGetProcedureAddress NtCreateSection NtClose NtMapViewOfSection NtReadFile CryptEncrypt SetFilePointer NtOpenSection NtCreateFile NtClose NtCreateSection NtClose NtMapViewOfSection LdrGetProcedureAddress LdrGetProcedureAddress GetSystemTimeAsFileTime NtOpenSection NtMapViewOfSection LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress GetSystemTimeAsFileTime GetSystemTimeAsFileTime GetSystemTimeAsFileTime GetSystemTimeAsFileTime GetSystemTimeAsFileTime GetSystemTimeAsFileTime GetSystemTimeAsFileTime NtClose NtClose NtUnmapViewOfSection GetSystemTimeAsFileTime GetFileAttributesExW NtOpenFile NtQueryInformationFile NtAllocateVirtualMemory NtAllocateVirtualMemory NtReadFile NtFreeVirtualMemory NtClose NtOpenFile NtQueryInformationFile NtAllocateVirtualMemory NtAllocateVirtualMemory NtReadFile NtFreeVirtualMemory NtClose NtOpenFile NtQueryInformationFile NtAllocateVirtualMemory NtAllocateVirtualMemory NtReadFile NtFreeVirtualMemory NtClose LdrGetProcedureAddress CoUninitialize GetSystemTimeAsFileTime RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey NtWriteFile RegOpenKeyExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegCloseKey RegCloseKey RegCloseKey CoInitializeEx CoUninitialize GetSystemTimeAsFileTime LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW LdrGetProcedureAddress RegQueryValueExW LdrGetProcedureAddress RegCloseKey LdrGetProcedureAddress SetFilePointer LdrGetProcedureAddress LdrGetProcedureAddress CoCreateInstance NtOpenSection NtOpenSection NtOpenSection NtMapViewOfSection NtWriteFile NtOpenSection NtMapViewOfSection GetSystemDirectoryW GetFileAttributesExW NtOpenSection NtOpenSection NtOpenSection NtMapViewOfSection NtOpenSection NtMapViewOfSection GetSystemDirectoryW GetFileAttributesExW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegOpenKeyExW RegCloseKey RegQueryValueExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExA RegOpenKeyExW RegCloseKey LdrGetDllHandle LdrGetDllHandle RegOpenKeyExW RegQueryValueExW RegCloseKey NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW FindFirstFileExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey FindFirstFileExW RegOpenKeyExW NtQueryDirectoryFile RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey NtOpenSection NtMapViewOfSection RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegOpenKeyExW RegCloseKey RegQueryValueExW RegQueryValueExW LdrGetProcedureAddress RegCloseKey RegCloseKey LdrGetProcedureAddress NtOpenKey NtQueryValueKey NtClose LdrGetProcedureAddress NtOpenKey NtQueryValueKey NtClose RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey SHGetFolderPathW SetErrorMode GetFileAttributesW SetErrorMode NtCreateFile GetSystemInfo GetFileSizeEx NtCreateSection NtMapViewOfSection NtUnmapViewOfSection NtClose NtClose RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey SHGetFolderPathW SetErrorMode GetFileAttributesW SetErrorMode RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW NtCreateFile NtOpenFile RegOpenKeyExW NtQueryDirectoryFile NtClose NtQueryDirectoryFile RegOpenKeyExW RegOpenKeyExW RegCloseKey RegCloseKey RegOpenKeyExW FindFirstFileExW NtAllocateVirtualMemory RegOpenKeyExW RegOpenKeyExW NtQueryDirectoryFile RegCloseKey RegOpenKeyExW RegCloseKey NtCreateFile NtOpenFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile RegOpenKeyExW NtCreateFile RegOpenKeyExW RegCloseKey RegCloseKey RegOpenKeyExW RegOpenKeyExW NtOpenFile NtQueryDirectoryFile NtClose RegOpenKeyExW GetForegroundWindow RegOpenKeyExW FindFirstFileExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW NtQueryDirectoryFile RegOpenKeyExW RegQueryValueExW NtCreateFile RegQueryValueExW NtOpenFile GetVolumeNameForVolumeMountPointW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW NtCreateFile RegCloseKey RegOpenKeyExW NtOpenFile RegOpenKeyExW RegCloseKey NtQueryDirectoryFile NtClose RegQueryValueExW RegCloseKey NtClose NtQueryDirectoryFile NtClose FindFirstFileExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW LoadStringW GetSystemWindowsDirectoryW GetSystemDirectoryW RegOpenKeyExW LdrLoadDll LdrGetProcedureAddress RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey NtCreateMutant NtCreateMutant GetNativeSystemInfo GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile LdrLoadDll LdrGetProcedureAddress LdrLoadDll NtQuerySystemInformation RegOpenKeyExA RegQueryValueExA RegCloseKey NtOpenKey NtQueryValueKey NtClose NtOpenKey NtQueryValueKey NtClose NtOpenKey NtQueryValueKey NtClose RegOpenKeyExW GlobalMemoryStatusEx NtOpenKey NtQueryValueKey NtOpenKey NtQueryValueKey NtClose NtClose NtDuplicateObject NtAllocateVirtualMemory NtAllocateVirtualMemory LdrGetProcedureAddress NtDuplicateObject NtAllocateVirtualMemory NtAllocateVirtualMemory GetVolumeNameForVolumeMountPointW NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey NtDuplicateObject NtClose GetVolumeNameForVolumeMountPointW NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey NtClose RegQueryValueExW RegCloseKey NtDuplicateObject GetVolumeNameForVolumeMountPointW NtClose NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose LdrGetProcedureAddress LdrUnloadDll CreateProcessInternalW NtClose NtClose NtClose LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress RegCloseKey RegCloseKey RegCloseKey RegCloseKey ShellExecuteExW GetTempPathW FindResourceA SizeofResource LoadResource NtCreateFile SetFilePointer NtWriteFile NtClose NtClose LdrGetProcedureAddress CoUninitialize LdrUnloadDll NtClose NtClose NtClose SearchPathW SearchPathW NtCreateFile NtCreateSection NtMapViewOfSection GetFileSize GetSystemDirectoryW NtOpenKey NtQueryValueKey LdrLoadDll NtClose NtAllocateVirtualMemory NtAllocateVirtualMemory GetSystemInfo LdrLoadDll LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory exception NtFreeVirtualMemory NtFreeVirtualMemory NtFreeVirtualMemory NtFreeVirtualMemory NtFreeVirtualMemory NtUnmapViewOfSection NtClose NtClose NtAllocateVirtualMemory SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtAllocateVirtualMemory NtSetInformationFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtAllocateVirtualMemory NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtAllocateVirtualMemory NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtAllocateVirtualMemory NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFil"]
#txt = [" ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrloaddll ldrgetprocedureaddress ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress regopenkeyexa regopenkeyexa regopenkeyexa ntopenkey ntqueryvaluekey ntclose ntopenkey ntqueryvaluekey ntclose ntclose ntqueryattributesfile ntqueryattributesfile ntqueryattributesfile ntqueryattributesfile loadstringa ntallocatevirtualmemory ntallocatevirtualmemory loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa loadstringa ldrgetdllhandle ldrgetprocedureaddress ldrgetdllhandle ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress ldrgetprocedureaddress findfirstfileexw ntqueryvaluekey ldrloaddll ldrgetprocedureaddress ldrgetprocedureaddress getsystemwindowsdirectoryw ntcreatefile ntcreatesection ntmapviewofsection ntclose ntclose regopenkeyexa regqueryvalueexa regqueryvalueexa regclosekey ntcreatefile getfiletype ntwritefile ntwritefile ntclose copyfilea findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile ntcreatefile ntreadfile ntwritefile ntclose ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntclose deletefilew findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile ntcreatefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntclose ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntclose deletefilew findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile ntcreatefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntclose ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntclose deletefilew findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile ntcreatefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntclose ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntclose deletefilew findfirstfileexw ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile ntcreatefile ntreadfile ntwritefile ntclose ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntclose deletefilew ntquerydirectoryfile findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile ntcreatefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntclose ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntclose deletefilew findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntquerydirectoryfile ntclose ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile ntcreatefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntclose ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntclose deletefilew findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile ntcreatefile ntreadfile ntwritefile ntclose ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntclose deletefilew findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose findfirstfileexw ntclose ntquerydirectoryfile findfirstfileexw ntallocatevirtualmemory ntquerydirectoryfile ntquerydirectoryfile ntclose ntquerydirectoryfile ntclose ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile ntquerydirectoryfile ntclose findfirstfileexw ntquerydirectoryfile findfirstfileexw ntquerydirectoryfile findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile ntcreatefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntclose ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntclose deletefilew findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile ntcreatefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntreadfile ntwritefile ntclose ntclose findfirstfileexw ntclose ntcreatefile ntclose ntcreatefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile setfilepointer setfilepointer setfilepointer setfilepointer ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile ntwritefile LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress NtProtectVirtualMemory NtProtectVirtualMemory FindResourceA SizeofResource LoadResource NtOpenDirectoryObject NtOpenMutant NtCreateMutant LdrLoadDll LdrGetProcedureAddress CryptAcquireContextA LdrGetProcedureAddress CryptGenKey LdrGetProcedureAddress CryptExportKey CryptExportKey LdrGetProcedureAddress NtAllocateVirtualMemory CryptAcquireContextA LdrGetProcedureAddress NtAllocateVirtualMemory LdrGetProcedureAddress CryptEncrypt LdrGetProcedureAddress LdrGetProcedureAddress CreateThread SetErrorMode NtDelayExecution FindFirstFileExW NtClose NtOpenKey NtQueryValueKey NtClose NtOpenKey NtQueryValueKey NtClose NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer SHGetFolderPathW NtCreateFile SetFilePointer NtWriteFile NtWriteFile NtClose RegOpenKeyExA NtReadFile CryptEncrypt SetFilePointer LdrLoadDll LdrGetProcedureAddress OleInitialize LdrGetProcedureAddress LdrGetProcedureAddress SetErrorMode GetFileAttributesW SetErrorMode GetSystemDirectoryW RegOpenKeyExA RegOpenKeyExA LdrLoadDll LdrGetProcedureAddress LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW LdrGetProcedureAddress CreateActCtxW GetSystemWindowsDirectoryW CreateActCtxW LdrGetDllHandle LdrGetProcedureAddress LdrLoadDll LdrLoadDll LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW LdrLoadDll LdrGetProcedureAddress RegEnumKeyW RegOpenKeyExW RegQueryValueExW RegCloseKey RegEnumKeyW RegCloseKey LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW NtWriteFile SetFilePointer RegOpenKeyExW LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW NtWriteFile NtClose NtOpenFile NtQueryInformationFile RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW NtSetInformationFile NtClose NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW FindFirstFileExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegOpenKeyExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW NtReadFile CryptEncrypt SetFilePointer RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW NtWriteFile SetFilePointer RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW NtWriteFile NtClose NtOpenFile NtQueryInformationFile RegQueryValueExW RegCloseKey RegCloseKey RegCloseKey RegCloseKey LdrGetProcedureAddress CoInitializeEx LdrGetProcedureAddress CoCreateInstance NtOpenSection NtOpenSection NtSetInformationFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile CreateDirectoryW NtCreateFile LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegQueryValueExW RegQueryValueExW GetSystemTimeAsFileTime GetSystemTimeAsFileTime RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW NtQueryDirectoryFile NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey LdrLoadDll LdrGetProcedureAddress RegCloseKey NtClose LdrGetProcedureAddress NtCreateSection NtClose NtMapViewOfSection NtReadFile CryptEncrypt SetFilePointer NtOpenSection NtCreateFile NtClose NtCreateSection NtClose NtMapViewOfSection LdrGetProcedureAddress LdrGetProcedureAddress GetSystemTimeAsFileTime NtOpenSection NtMapViewOfSection LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress GetSystemTimeAsFileTime GetSystemTimeAsFileTime GetSystemTimeAsFileTime GetSystemTimeAsFileTime GetSystemTimeAsFileTime GetSystemTimeAsFileTime GetSystemTimeAsFileTime NtClose NtClose NtUnmapViewOfSection GetSystemTimeAsFileTime GetFileAttributesExW NtOpenFile NtQueryInformationFile NtAllocateVirtualMemory NtAllocateVirtualMemory NtReadFile NtFreeVirtualMemory NtClose NtOpenFile NtQueryInformationFile NtAllocateVirtualMemory NtAllocateVirtualMemory NtReadFile NtFreeVirtualMemory NtClose NtOpenFile NtQueryInformationFile NtAllocateVirtualMemory NtAllocateVirtualMemory NtReadFile NtFreeVirtualMemory NtClose LdrGetProcedureAddress CoUninitialize GetSystemTimeAsFileTime RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey NtWriteFile RegOpenKeyExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegQueryValueExW RegCloseKey RegCloseKey RegCloseKey CoInitializeEx CoUninitialize GetSystemTimeAsFileTime LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW LdrGetProcedureAddress RegQueryValueExW LdrGetProcedureAddress RegCloseKey LdrGetProcedureAddress SetFilePointer LdrGetProcedureAddress LdrGetProcedureAddress CoCreateInstance NtOpenSection NtOpenSection NtOpenSection NtMapViewOfSection NtWriteFile NtOpenSection NtMapViewOfSection GetSystemDirectoryW GetFileAttributesExW NtOpenSection NtOpenSection NtOpenSection NtMapViewOfSection NtOpenSection NtMapViewOfSection GetSystemDirectoryW GetFileAttributesExW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LoadStringW LdrGetProcedureAddress LdrGetProcedureAddress RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegOpenKeyExW RegCloseKey RegQueryValueExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExA RegOpenKeyExW RegCloseKey LdrGetDllHandle LdrGetDllHandle RegOpenKeyExW RegQueryValueExW RegCloseKey NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey LdrGetProcedureAddress RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW FindFirstFileExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey FindFirstFileExW RegOpenKeyExW NtQueryDirectoryFile RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey NtOpenSection NtMapViewOfSection RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegOpenKeyExW RegCloseKey RegQueryValueExW RegQueryValueExW LdrGetProcedureAddress RegCloseKey RegCloseKey LdrGetProcedureAddress NtOpenKey NtQueryValueKey NtClose LdrGetProcedureAddress NtOpenKey NtQueryValueKey NtClose RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey SHGetFolderPathW SetErrorMode GetFileAttributesW SetErrorMode NtCreateFile GetSystemInfo GetFileSizeEx NtCreateSection NtMapViewOfSection NtUnmapViewOfSection NtClose NtClose RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegQueryValueExW RegCloseKey SHGetFolderPathW SetErrorMode GetFileAttributesW SetErrorMode RegOpenKeyExW RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW NtCreateFile NtOpenFile RegOpenKeyExW NtQueryDirectoryFile NtClose NtQueryDirectoryFile RegOpenKeyExW RegOpenKeyExW RegCloseKey RegCloseKey RegOpenKeyExW FindFirstFileExW NtAllocateVirtualMemory RegOpenKeyExW RegOpenKeyExW NtQueryDirectoryFile RegCloseKey RegOpenKeyExW RegCloseKey NtCreateFile NtOpenFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile RegOpenKeyExW NtCreateFile RegOpenKeyExW RegCloseKey RegCloseKey RegOpenKeyExW RegOpenKeyExW NtOpenFile NtQueryDirectoryFile NtClose RegOpenKeyExW GetForegroundWindow RegOpenKeyExW FindFirstFileExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW RegOpenKeyExW NtQueryDirectoryFile RegOpenKeyExW RegQueryValueExW NtCreateFile RegQueryValueExW NtOpenFile GetVolumeNameForVolumeMountPointW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW NtCreateFile RegCloseKey RegOpenKeyExW NtOpenFile RegOpenKeyExW RegCloseKey NtQueryDirectoryFile NtClose RegQueryValueExW RegCloseKey NtClose NtQueryDirectoryFile NtClose FindFirstFileExW RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW LoadStringW GetSystemWindowsDirectoryW GetSystemDirectoryW RegOpenKeyExW LdrLoadDll LdrGetProcedureAddress RegQueryValueExW RegCloseKey RegOpenKeyExW RegQueryValueExW RegCloseKey NtCreateMutant NtCreateMutant GetNativeSystemInfo GetSystemWindowsDirectoryW GetSystemWindowsDirectoryW FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile LdrLoadDll LdrGetProcedureAddress LdrLoadDll NtQuerySystemInformation RegOpenKeyExA RegQueryValueExA RegCloseKey NtOpenKey NtQueryValueKey NtClose NtOpenKey NtQueryValueKey NtClose NtOpenKey NtQueryValueKey NtClose RegOpenKeyExW GlobalMemoryStatusEx NtOpenKey NtQueryValueKey NtOpenKey NtQueryValueKey NtClose NtClose NtDuplicateObject NtAllocateVirtualMemory NtAllocateVirtualMemory LdrGetProcedureAddress NtDuplicateObject NtAllocateVirtualMemory NtAllocateVirtualMemory GetVolumeNameForVolumeMountPointW NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey NtDuplicateObject NtClose GetVolumeNameForVolumeMountPointW NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey NtClose RegQueryValueExW RegCloseKey NtDuplicateObject GetVolumeNameForVolumeMountPointW NtClose NtClose RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey RegOpenKeyExW RegOpenKeyExW RegCloseKey RegQueryValueExW RegCloseKey GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW GetVolumePathNamesForVolumeNameW NtClose LdrGetProcedureAddress LdrUnloadDll CreateProcessInternalW NtClose NtClose NtClose LdrGetDllHandle LdrGetProcedureAddress LdrGetProcedureAddress RegCloseKey RegCloseKey RegCloseKey RegCloseKey ShellExecuteExW GetTempPathW FindResourceA SizeofResource LoadResource NtCreateFile SetFilePointer NtWriteFile NtClose NtClose LdrGetProcedureAddress CoUninitialize LdrUnloadDll NtClose NtClose NtClose SearchPathW SearchPathW NtCreateFile NtCreateSection NtMapViewOfSection GetFileSize GetSystemDirectoryW NtOpenKey NtQueryValueKey LdrLoadDll NtClose NtAllocateVirtualMemory NtAllocateVirtualMemory GetSystemInfo LdrLoadDll LdrGetProcedureAddress LdrLoadDll LdrGetProcedureAddress NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory exception NtFreeVirtualMemory NtFreeVirtualMemory NtFreeVirtualMemory NtFreeVirtualMemory NtFreeVirtualMemory NtUnmapViewOfSection NtClose NtClose NtAllocateVirtualMemory SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtAllocateVirtualMemory NtSetInformationFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtAllocateVirtualMemory NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtAllocateVirtualMemory NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtAllocateVirtualMemory NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtCreateFile GetFileSize SetFilePointer NtReadFile CryptEncrypt SetFilePointer NtWriteFile SetFilePointer NtWriteFile NtClose NtOpenFile NtQueryInformationFile NtSetInformationFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFileExW NtQueryDirectoryFile NtQueryDirectoryFile NtClose FindFirstFil"]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = [' Cryptodef', ' Loktrom ', ' Loktrom', ' Deshacop', ' Gpcode']
print(pred, labels[np.argmax(pred)])



[[0.00122389 0.17437693 0.6184836  0.03414536 0.16685629 0.00491386]]  Loktrom


In [23]:

p[0]=articles[0]
print (p)

IndexError: list assignment index out of range

In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_labels)
valid_y = encoder.fit_transform(validation_labels)